# Agile Issue Generator

This Python script is designed to help automate the creation and uploading of Agile issues, such as Epics, Stories, and Subtasks, to either Jira or Azure DevOps (DevOps). It generates a structured list of project tasks based on the provided project description, and you can customize the number of epics, stories, and subtasks to be generated.

## Prerequisites

Before using this script, ensure you have the following prerequisites:

1. **Python**: You must have Python installed on your machine. You can download it from [python.org](https://www.python.org/downloads/).

2. **Jira or Azure DevOps Account**: Depending on your choice, you should have an account and API access to either Jira or Azure DevOps.

3. **API Key or Personal Access Token (PAT)**: You'll need an API key for Jira or a PAT for Azure DevOps for authentication.

4. **Excel File**: If you're using Azure DevOps, you need an Excel file containing the issue data.

## Setup

1. Clone this repository or download the script.

2. Open the script in your preferred code editor.

3. Modify the script's variables to match your project's requirements:

   ```python
   num_epics = 1
   num_stories = 6
   num_subtasks = 0 # You can change this number as needed
   project_name = ""
   root_name = ''


In [1]:
# Required libraries and modules
from dotenv import load_dotenv
import pandas as pd
import os
import json
import requests
from requests.auth import HTTPBasicAuth
import hashlib
import openai
from azure.devops.connection import Connection
from azure.devops.v7_0.work_item_tracking.models import JsonPatchOperation, WorkItemRelation
from msrest.authentication import BasicAuthentication

# Configuration class to load environment variables
class Config:
    def __init__(self, path='./environment.env'):
        # Load environment variables from specified file path
        load_dotenv(dotenv_path=path)
        # Jira API configurations
        self.jira_id = os.getenv("JIRA_ID")
        self.jira_key = os.getenv("JIRA_KEY")
        # OpenAI API configurations
        self.open_ai_endpoint = os.getenv("OPENAI_ENDPOINT")
        self.openai_key = os.getenv("OPENAI_KEY")
        self.chat_engine = os.getenv("CHAT_ENGINE")

# Class to handle Jira API operations
class Jira():
    def __init__(self, config):
        # Initialize with Jira configurations
        self.username = config.jira_id
        self.api_token = config.jira_key
        self.url = self.username.split('@')[0]

    # Method to fetch projects from Jira
    def fetch_projects(self, url):
        response = requests.get(url, auth=HTTPBasicAuth(self.username, self.api_token))
        return response.json() if response.status_code == 200 else None

    # Method to retrieve issues for a specific project and issue type from Jira
    def get_issues(self, project_key, project_name, issue_type):
        url_issues = f"https://{self.url}.atlassian.net/rest/api/3/search?jql=project={project_key} AND issuetype='{issue_type}'"
        response = requests.get(url_issues, auth=HTTPBasicAuth(self.username, self.api_token))
        issues_json = []
        if response.status_code == 200:
            issues = response.json().get('issues', [])
            for issue in issues:
                issue_fields = issue['fields']
                issue_name = issue_fields['summary']
                description_content = issue_fields['description']
                # Extract text from the issue's description content
                description_texts = self.extract_text(description_content)
                description_text = " ".join(description_texts)
                parent = issue_fields.get('parent', {}).get('fields', {}).get('summary') if issue_type != 'Epic' else None
                current_hierarchy = f"Parent: {parent}" if parent else "" + f" / Current: {issue_name}"
                issue_json = {
                    "project_name": project_name,
                    "name": issue_name,
                    "type": issue_type,
                    "hierarchy": current_hierarchy,
                    "description": description_text
                }
                issues_json.append(issue_json)
        return issues_json
    
    # New method to get all issue IDs for a given project
    def get_all_issue_ids(self, project_key):
        url_issues = f"https://{self.url}.atlassian.net/rest/api/3/search?jql=project={project_key}"
        response = requests.get(url_issues, auth=HTTPBasicAuth(self.username, self.api_token))
        if response.status_code == 200:
            issues = response.json().get('issues', [])
            return [issue['id'] for issue in issues]
        return []

    # New method to delete an issue given its ID
    def delete_issue(self, issue_id):
        url_delete = f"https://{self.url}.atlassian.net/rest/api/3/issue/{issue_id}"
        response = requests.delete(url_delete, auth=HTTPBasicAuth(self.username, self.api_token))
        return response.status_code == 204

    # New method to delete all issues in a project
    def delete_all_issues(self, project_key):
        issue_ids = self.get_all_issue_ids(project_key)
        for issue_id in issue_ids:
            self.delete_issue(issue_id)
        return f"All issues in project {project_key} have been deleted."

    # Recursive method to extract text from Jira's description content (which can be nested)
    def extract_text(self, content):
        texts = []
        if isinstance(content, dict):
            for key, value in content.items():
                if key == 'text':
                    texts.append(value)
                elif isinstance(value, (list, dict)):
                    texts.extend(self.extract_text(value))
        elif isinstance(content, list):
            for item in content:
                texts.extend(self.extract_text(item))
        return texts

    # Static method to modify issue names by prefixing them with the issue type
    @staticmethod
    def modify_issue_names(all_issues_json):
        for issue in all_issues_json:
            issue_type = issue.get('type', '')
            issue_name = issue.get('name', '')
            issue['name'] = f"{issue_type}: {issue_name}"
        return all_issues_json
    # Static method to generate and add a unique key for each issue based on its name
    @staticmethod
    def add_key_field(all_issues_json):
        for issue in all_issues_json:
            name = issue['name']
            hash_object = hashlib.sha256(name.encode())
            key = int(hash_object.hexdigest(), 16) % (10**10)
            issue['key'] = str(key)
        return all_issues_json
    
    def create_new_issue(self, project_key, issue_name, issue_type, description, parent=None):
        url_create_issue = "https://{self.url}.atlassian.net/rest/api/3/issue"
        # Payload for creating an issue
        payload = {
            "fields": {
                "project": {
                    "key": project_key
                },
                "summary": issue_name,
                "issuetype": {
                    "name": issue_type
                },
                "description": {
                    "type": "doc",
                    "version": 1,
                    "content": [
                        {
                            "type": "paragraph",
                            "content": [
                                {
                                    "text": description,
                                    "type": "text"
                                }
                            ]
                        }
                    ]
                }
            }
        }

        # If a parent is provided and the issue type is not an Epic, set the parent field
        if parent and issue_type.lower() != 'epic':
            payload['fields']['parent'] = {"key": parent}
            
        print(f"Payload for {issue_name}: {payload}")  # Debugging print statement
        response = requests.post(url_create_issue, auth=HTTPBasicAuth(self.username, self.api_token), json=payload)
        print(f"Response for {issue_name}: {response.json()}")  # Debugging print statement
        return response.json()

    def create_issues_from_csv(self, csv_file_path, project_key):
        df = pd.read_csv(csv_file_path)
        created_issues = []
        parent_mapping = {}  # To store parent issue keys

        for _, row in df.iterrows():
            issue_name = row['title']
            description = row['description']
            issue_type = row['type']
            parent_name = row['parent']
            skills = row['skills'] if row['skills'] else None
            roles = row['roles']if row['roles'] else None
            concat_description = f"Description:\n{description}\nSkills_Requried:\n{skills}\nRoles:\n{roles}\n"


            # Determine the parent key for Stories and Subtasks
            parent_key = None
            if parent_name and issue_type.lower() != 'epic':
                parent_key = parent_mapping.get(parent_name)
                if not parent_key:
                    print(f"Parent key not found for {issue_name}. Parent Name: {parent_key}")
                    continue

            # Create the issue and retrieve its key
            response = self.create_new_issue(project_key, issue_name, issue_type, concat_description, parent_key)
            if response and 'key' in response:
                issue_key = response['key']
                created_issues.append(issue_key)
                # Store the key of Epics and Stories for their children to reference
                if issue_type.lower() in ['epic', 'story']:
                    parent_mapping[issue_name] = issue_key
            else:
                print(f"Failed to create issue: {issue_name}")

        return created_issues

class openai_issues:
    def __init__(self, config):
        self.api_key = config.openai_key
        self.api_version = "2023-07-01-preview"
        self.api_type = "azure"
        self.api_base = config.open_ai_endpoint
        self.engine = config.chat_engine
        # Set up the global API configuration
        openai.api_key = self.api_key
        openai.api_version = self.api_version
        openai.api_type = self.api_type
        openai.api_base = self.api_base

    def call_openai(self, messages, function_schema=None, function_call=None):
        
        kwargs = {
        "engine": self.engine,
        "messages": messages,
        "temperature": 0.5
        }
        # Include function schema and call if they are provided
        if function_schema is not None:
            kwargs["functions"] = function_schema
        if function_call is not None:
            kwargs["function_call"] = function_call

        response = openai.ChatCompletion.create(**kwargs)
        
        return response
    
    def generate_project_tasks_to_csv(self, project_description, output, schema):
        persona = """You are an Expert Technology Architect. Your job is to generate a CSV with columns that represent a Project in Jira, based on your expert knowledge of past projects of a similar type. The Columns of the CSV are [title,description,parent,type]. You must understand the project's requirements, organize the project in Epics, Stories and Subtasks. The title will contain the name of the issue, the description must be a detailed AGILE compliant Epic, Story or Subtask (50 words or less). Depending on the complexity of the user task, you may have one or multiple Stories / Subtasks under an Epic. Subtasks are part of Stories, and Stories are part of Epics. The final JSON (if converted to a Jira Project), should allow a team to build the project described by the user."""
    
        # Make the API call to OpenAI
        response = self.call_openai(
            messages=[
                {"role": "system", "content": persona},
                {"role": "user", "content": project_description}
            ],
            function_schema=schema,
            function_call={"name": "generate_project_JIRA"},
        )
        
        # Process the response and export to CSV
        try:
            dirty_class = response['choices'][0]['message']['function_call']['arguments']
            task_data = json.loads(dirty_class).get("issues")
        except json.JSONDecodeError:
            print("Failed to decode JSON from response")
            return
        
        df = pd.DataFrame(task_data)
        df.loc[df['type'] == 'Epic', 'parent'] = None

        df.to_csv(output, index=False, encoding='utf-8')
        print(f"CSV file '{output}' has been created in the current working directory.")

    def generate_description(self, project_context, current_title, issue_type, other_items_context, current_description):
        # Define the length of the description based on issue type
        length = 300 if issue_type == "Epic" else 200 if issue_type == "Story" else 100

        # Construct the prompt with additional context
        prompt = (
            f"Project Overview:\n{project_context}\n\n"
            f"Other Items in the Project:\n{other_items_context}\n\n"
            f"Write an Agile Compliant Description of length: {length} for a(n) {issue_type} titled: '{current_title}' - {current_description}"
            f"If it's an Epic, you must assume the persona of a Product Owner, if it's a Story, assume a technical lead, and a developer writes the subtasks."
        )
        messages = [
            {"role": "system", "content": "You are a Master Solutions Architect, both a technical expert and a business leader."},
            {"role": "user", "content": prompt}
        ]
        # Call OpenAI API
        response = self.call_openai(
            messages=messages
        )
        # Return the enhanced description extracted from the response
        return response['choices'][0]['message']['content']
    
    def update_project_descriptions(self, csv_file_path, project_description, output_file_name):
        project_context = project_description
        
        df = pd.read_csv(csv_file_path)
        other_items_context = "\n".join([f"{row['type']}: {row['title']}" for _, row in df.iterrows()])

        for index, row in df.iterrows():
            # Skipping the API request simulation, directly calling the method to generate descriptions
            current_title = row['title']
            issue_type = row['type']
            current_description = row['description']
            context_without_current = other_items_context.replace(f"{issue_type}: {current_title}\n", "")
            enhanced_description = self.generate_description(
                project_context, current_title, issue_type, context_without_current, current_description
            )
            df.at[index, 'description'] = enhanced_description

        df.to_csv(output_file_name, index=False)   

    def generate_skills_for_issues(self, csv_file_name, schema):
        df = pd.read_csv(csv_file_name)
        df['skills'] = None
        for index, row in df.iterrows():
            title, description, issue_type = row['title'], row['description'], row['type']
            parent = row['parent'] if pd.notna(row['parent']) else "None"
            prompt = f"Given a JIRA issue with the title '{title}', description '{description}', type '{issue_type}', and parent '{parent}', list the skills required to successfully accomplish this issue in the shape of a list."
            messages = [
                {"role": "system", "content": "You are a Master Solutions Architect, both a technical expert and a business leader."},
                {"role": "user", "content": f"{prompt}"}
            ]
            response = self.call_openai(
                messages=messages,
                function_schema=schema,
                function_call={"name": "generate_skills"}
            )
            
            try:
                # Extract the skills from the API response and clean it up using regex
                dirty_class = response['choices'][0]['message']['function_call']['arguments']
                task_data = json.loads(dirty_class).get("skills")
            except json.JSONDecodeError:
                print("Failed to decode JSON from response")
                continue  # Move on to the next iteration

            df.at[index, 'skills'] = task_data

        # Generate new filename with '_skills'
        base_name, ext = os.path.splitext(csv_file_name)
        new_filename = f"{base_name}_skills{ext}"
        df.to_csv(new_filename, index=False)

        return new_filename

    def generate_roles_for_issues(self, csv_file_name, schema):
        df = pd.read_csv(csv_file_name)
        df['roles'] = None  # Add a new column for roles if it doesn't exist

        for index, row in df.iterrows():
            title = row['title']
            description = row['description']
            issue_type = row['type']
            parent = row['parent'] if pd.notna(row['parent']) else "None"
            
            # Craft the prompt and call OpenAI API
            response = self.call_openai(
                messages=[
                    {"role": "system", "content": "You are a Master Solutions Architect, both a technical expert and a business leader."},
                    {"role": "user", "content": f"Given a JIRA issue with the title '{title}', description '{description}', type '{issue_type}', and parent '{parent}', list 5 employee roles in an IT company that could successfully deliver this Issue."}
                ],
                function_schema=schema,
                function_call={"name": "generate_roles"}
            )
            
            try:
                # Extract the roles from the API response and clean it up using regex
                dirty_class = response['choices'][0]['message']['function_call']['arguments']
                task_data = json.loads(dirty_class).get("roles")
            except json.JSONDecodeError:
                print("Failed to decode JSON from response")
                continue  # Move on to the next iteration

            df.at[index, 'roles'] = task_data
                # Generate new filename with '_roles'
        base_name, ext = os.path.splitext(csv_file_name)
        new_filename = f"{base_name}_roles{ext}"
        df.to_csv(new_filename, index=False)
        print(f"roles and Skills CSV '{new_filename}' has been created in the current working directory.")
        return new_filename

def main_jira_csv(root_name, project_name, project_description, config, csv_schema, skills_schema, role_schema):
    # Create an instance of openai_jira with the provided configuration
    jira_api = openai_issues(config)

    # Define output CSV schema
    output_schema = root_name
    PROJECT = project_name # To be used if you want to upload csv into Jira as a Project
    file_name = f"{output_schema.replace('.csv', '')}_details.csv"
    
    # Run the process
    jira_api.generate_project_tasks_to_csv(project_description, output_schema, csv_schema)
    jira_api.update_project_descriptions(output_schema, project_description, file_name)
    skills_df = jira_api.generate_skills_for_issues(file_name, skills_schema)
    final_csv = jira_api.generate_roles_for_issues(skills_df, role_schema)
    
    return final_csv

class AzureDevOpsSDK:
    def __init__(self, organization, project, personal_access_token):
        self.organization = organization
        self.project = project
        self.credentials = BasicAuthentication('', personal_access_token)
        self.connection = Connection(base_url=f'https://dev.azure.com/{self.organization}', 
                                     creds=self.credentials)

    def create_work_item(self, work_item_type, title, description):
        work_item_tracking_client = self.connection.clients.get_work_item_tracking_client()
        patch_document = [
            JsonPatchOperation(op="add", path="/fields/System.Title", value=title),
            JsonPatchOperation(op="add", path="/fields/System.Description", value=description)
        ]
        work_item = work_item_tracking_client.create_work_item(document=patch_document, 
                                                               project=self.project, 
                                                               type=work_item_type)
        return work_item

    def link_parent(self, child_id, parent_id):
        work_item_tracking_client = self.connection.clients.get_work_item_tracking_client()
        patch_document = [
            JsonPatchOperation(
                op="add",
                path="/relations/-",
                value={
                    "rel": "System.LinkTypes.Hierarchy-Reverse",
                    "url": f"https://dev.azure.com/{self.organization}/_apis/wit/workItems/{parent_id}"
                }
            )
        ]
        work_item_tracking_client.update_work_item(document=patch_document, id=child_id)

    def create_and_link_work_items(self, excel_file_path):
        df = pd.read_excel(excel_file_path)
        work_items_map = {}

    # Create all work items first and store their IDs
        for _, row in df.iterrows():
            title = row['title']
            description = row['description']
            work_item_type = row['type']

            work_item = self.create_work_item(work_item_type, title, description)
            work_items_map[title] = work_item.id

        # Link parent work items
        for _, row in df.iterrows():
            child_title = row['title']
            parent_title = row.get('parent')
            if pd.notna(parent_title) and parent_title in work_items_map and child_title != parent_title:
                child_id = work_items_map[child_title]
                parent_id = work_items_map[parent_title]
                self.link_parent(child_id, parent_id)

        return work_items_map

In [6]:
# Define variables for the number of epics, stories, and subtasks
num_epics = 1
num_stories = 6
num_subtasks = 0 # You can change this number as needed
project_name = ""
root_name = ''

# Create the project description using f-strings
project_description = f""" 


"""
csv_schema = [
    {
        "name": "generate_project_JIRA",
        "description": "Generate a structured list of project tasks including Epics, Stories, and Subtasks based on the given project description. Each task should include 'title', 'description', 'parent', and 'type'. Epics have 'None' as the parent.",
        "parameters": {
            "type": "object",
            "properties": {
                "issues": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "title": {"type": "string"},
                            "description": {"type": "string"},
                            "parent": {"type": ["string", "null"]},
                            "type": {"type": "string", "enum": ["Epic", "Story", "Subtask"]}
                        },
                        "required": ["title", "description", "type"]
                    }
                }
            },
            "required": ["issues"]
        }
    }
]
role_schema = [
    {
        "name": "generate_roles",
        "description": "Generates a list of roles in an IT Company that could successfully deliver the requirements of a given Jira Issue",
        "parameters": {
            "type": "object",
            "properties": {
                "roles": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "An array of roles in an IT Company that could successfully deliver the requirements of a given Jira Issue."
                }
            },
            "required": ["roles"]
        }
    }
]
skills_schema = [
    {
        "name": "generate_skills",
        "description": "Generates a list of skills required to successfully accomplish a JIRA issue depending on its requirements.",
        "parameters": {
            "type": "object",
            "properties": {
                "skills": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "An array of skills an employee would require to succesfully complete the requirements outlined in a Jira issue."
                }
            },
            "required": ["skills"]
        }
    }
]

final_df = main_jira_csv(root_name, project_name,project_description, Config(), csv_schema, skills_schema, role_schema)

### IF you have a Jira server and API key, you may run the code below to upload:
# Jira(Config()).create_issues_from_csv(final_df, "project_name")

### IF you have a DevOps workspace and API key, you may run the code below to upload:
organization = ''  # Replace with your organization name
project = ''            # Replace with your project name
personal_access_token = ''       # Replace with your PAT
excel_file_path = ''  # Replace with the path to your Excel file

azure_devops_sdk = AzureDevOpsSDK(organization, project, personal_access_token)
work_items_map = azure_devops_sdk.create_and_link_work_items(excel_file_path)

CSV file './JiraPT/Output/test.csv' has been created in the current working directory.
